In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
data = pd.read_feather(r'C:\Users\leoac\OneDrive - Università degli Studi di Milano\Unimi\Subjects\Labs\Football analytics\SPR - Project\Seasonal-Player-Rating\Project2-3 - match-seasonal index\match_data.feather')

In [58]:
for col in data.columns:
    print(col)

competition
team
full_name
cluster_label
gametime
player_id
game_id
team_id
competition_id
season
accurate_back_zone_pass
accurate_corners_intobox
accurate_cross
accurate_cross_nocorner
accurate_fwd_zone_pass
accurate_goal_kicks
accurate_keeper_throws
accurate_launches
accurate_layoffs
accurate_long_balls
accurate_pass
accurate_through_ball
accurate_throws
aerial_lost
aerial_won
att_bx_centre
att_obx_centre
att_bx_right
att_bx_left
att_corner
att_fastbreak
att_freekick_goal
att_freekick_target
att_freekick_total
att_freekick_miss
att_freekick_post
att_goal_high_centre
att_goal_high_left
att_goal_high_right
att_goal_low_centre
att_goal_low_left
att_goal_low_right
att_hd_goal
att_hd_miss
att_hd_post
att_hd_target
att_hd_total
att_ibox_blocked
att_ibox_goal
att_ibox_miss
att_ibox_post
att_ibox_target
att_ibox_own_goal
att_obox_own_goal
att_lf_goal
att_lf_target
att_lf_total
att_miss_high
att_miss_high_left
att_miss_high_right
att_miss_left
att_miss_right
att_obox_blocked
att_obox_goal
att

In [59]:
# Fill nans with 0
data.fillna(0, inplace=True)

C:\Users\leoac\AppData\Local\Temp\ipykernel_15804\121700303.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna(0, inplace=True)


In [60]:
data.goals

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
84674    1.0
84675    0.0
84676    0.0
84677    0.0
84678    0.0
Name: goals, Length: 84679, dtype: float64

In [61]:
# Drop not useful and sparse columns
DBINFO=['competition',
'team',
'full_name',
'cluster_label',
'gametime',
'player_id',
'competition_id',
'season',
'side',
'stat_type',
'created_at',
'updated_at']

METRICS_TO_DROP = [
'att_bx_centre',
'att_obx_centre',
'att_bx_right',
'att_bx_left',
'att_corner',
'att_fastbreak',
'att_freekick_goal',
'att_freekick_target',
'att_freekick_total',
'att_freekick_miss',
'att_freekick_post',
'att_goal_high_centre',
'att_goal_high_left',
'att_goal_high_right',
'att_goal_low_centre',
'att_goal_low_left',
'att_goal_low_right',
'att_hd_goal',
'att_hd_miss',
'att_hd_post',
'att_hd_target',
'att_hd_total',
'att_ibox_blocked',
'att_ibox_goal',
'att_ibox_miss',
'att_ibox_post',
'att_ibox_target',
'att_lf_goal',
'att_lf_target',
'att_lf_total',
'att_miss_high',
'att_miss_high_left',
'att_miss_high_right',
'att_miss_left',
'att_miss_right',
'att_obox_blocked',
'att_obox_goal',
'att_obox_miss',
'att_obox_post',
'att_obx_left',
'att_obx_right',
'att_obox_target',
'att_obxd_left',
'att_obxd_right',
'att_obp_goal',
'att_lg_centre',
'att_lg_left',
'att_lg_right',
'att_one_on_one',
'att_cmiss_high',
'att_cmiss_high_right',
'att_cmiss_high_left',
'att_cmiss_left',
'att_cmiss_right',
'att_openplay',
'att_pen_goal',
'att_pen_miss',
'att_pen_post',
'att_pen_target',
'att_post_high',
'att_post_left',
'att_post_right',
'att_rf_goal',
'att_rf_target',
'att_rf_total',
'att_setpiece',
'att_sv_high_centre',
'att_sv_high_left',
'att_sv_high_right',
'att_sv_low_centre',
'att_sv_low_left',
'att_sv_low_right',
'attempts_conceded_ibox',
'attempts_conceded_obox',
'attempts_ibox',
'attempts_obox',
'game_started',
'winning_goal',
'goals_set_pieces_faced',
'goals_big_chance_faced',
'opp_touches',
'team_touches',
'team_touches_attack',
'team_touches_buildup',
'team_touches_defensive',
'player_touches'
]

METRICS_GK = [
'clean_sheet',
'dive_catch',
'dive_save',
'gk_smother',
'goal_kicks',
'good_high_claim',
'keeper_pick_up',
'keeper_throws',
'saved_ibox',
'saved_obox',
'saves',
'six_second_violation',
'stand_catch',
'stand_save',
'penalty_faced',
'total_keeper_sweeper',
'accurate_keeper_sweeper',
'diving_save',
'goals_prevented',
'Goals_prevented_per_xGoT',
'set_pieces_faced',
'big_chance_faced'
]

# Goals are to consider or not?
METRICS_GOALS = [
'goals_conceded',
'goals_conceded_ibox',
'goals_conceded_obox',
'goals_openplay'
]

METRICS_ASSISTS = [
    'goal_assist',
    'goal_assist_setplay',
    'goal_assist_openplay',
    'goal_assist_intentional',
    'second_goal_assist',
    'assist_own_goal',
    'goal_assist_deadball'
]

filtered_data = data.drop([col for col in DBINFO + METRICS_TO_DROP + METRICS_GK + METRICS_GOALS + METRICS_ASSISTS], axis=1)

In [62]:
# Remove 0 variance features
filtered_data = filtered_data[filtered_data.std()[filtered_data.std() > 0].index]

In [77]:
team_data = (filtered_data.groupby(['game_id', 'team_id']).sum()).reset_index()

In [79]:
# Define a function to subtract the first row from the second and viceversa and return both the observations
def subtract_first_from_second(group):
    # Drop game_id and team_id to save calculations
    group.drop(['game_id','team_id'], axis=1, inplace=True)

    # Create a copy of the group to save the new calculations
    new_group = group.copy()

    # Do calculation
    new_group.iloc[0] = group.iloc[0] - group.iloc[1]
    new_group.iloc[1] = group.iloc[1] - group.iloc[0]

    return new_group 

# Group by 'game_id' and apply the custom function
dataset = team_data.groupby('game_id').apply(subtract_first_from_second).reset_index(drop=True)

C:\Users\leoac\AppData\Local\Temp\ipykernel_15804\3213883085.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset = team_data.groupby('game_id').apply(subtract_first_from_second).reset_index(drop=True)


In [80]:
# Create the label
win_target = dataset.goals.apply(lambda goals: 1 if goals > 0 else 0)
dataset['win'] = win_target
# Drop goals if you don't want to consider them as a feature
dataset.drop('goals', axis=1, inplace=True)

In [81]:
dataset

,accurate_back_zone_pass,accurate_corners_intobox,accurate_cross,accurate_cross_nocorner,accurate_fwd_zone_pass,accurate_goal_kicks,accurate_launches,accurate_layoffs,accurate_long_balls,accurate_pass,...,cutbacks_received,match_day,progressive_passes,prog_carries_final_third,offensive_aerials,gpi,opxG,successful_switches,opxA,win
0,8.0,-1.0,-2.0,-1.0,9.0,2.0,-2.0,6.0,-32.0,19.0,...,0.0,-1,2.0,1,-6.0,-4.0,-0.057283,-11.0,0.114656,0
1,-8.0,1.0,2.0,1.0,-9.0,-2.0,2.0,-6.0,32.0,-19.0,...,0.0,1,-2.0,-1,6.0,4.0,0.057283,11.0,-0.114656,0
2,51.0,0.0,-1.0,-1.0,202.0,1.0,-1.0,-1.0,0.0,254.0,...,0.0,-2,25.0,1,-14.0,4.0,0.545908,1.0,1.427978,1
3,-51.0,0.0,1.0,1.0,-202.0,-1.0,1.0,1.0,0.0,-254.0,...,0.0,2,-25.0,-1,14.0,-4.0,-0.545908,-1.0,-1.427978,0
4,183.0,4.0,15.0,11.0,193.0,-2.0,-1.0,4.0,18.0,361.0,...,0.0,3,21.0,14,18.0,12.0,2.243139,17.0,2.538521,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5329,-171.0,1.0,-2.0,-2.0,-89.0,-1.0,-4.0,-8.0,-11.0,-258.0,...,0.0,-32,-10.0,-13,-2.0,-2.0,-1.164232,-8.0,-0.368811,0
5330,147.0,2.0,6.0,5.0,147.0,-1.0,-5.0,2.0,14.0,288.0,...,1.0,64,12.0,17,-2.0,5.0,0.288475,8.0,0.265980,0
5331,-147.0,-2.0,-6.0,-5.0,-147.0,1.0,5.0,-2.0,-14.0,-288.0,...,-1.0,-64,-12.0,-17,2.0,-5.0,-0.288475,-8.0,-0.265980,0
5332,65.0,1.0,-1.0,-2.0,-17.0,2.0,-1.0,11.0,-2.0,49.0,...,0.0,64,17.0,-5,0.0,-5.0,0.756004,5.0,0.198453,0


In [82]:
dataset.to_csv('dataset.csv', index=False)